#Intro
Penney's game is a 2 player nontransitive game that can be done using a deck of cards or filling some coins. It is needed that each binary state is roughly 50-50

Two players start by guessing a string of binary states such as HHT or TTT  
and the goal is to see who's string appears first. Each string is known to everyone


When first published, Penney's game uses a string of 3 heads or tails. It has also been shown that red-black cards of a deck of cards is approximate to the probabilities

#Goal
This python notebooks seeks 
1. To generalize the game to a string of length n by utilizing markov chains. 
2. To generalize the game to include m players

In [54]:
import numpy as np
import pandas as pd
import itertools as it

A markov chain can be represented as an adjacency matrix.  
I will be using binary to represent the states where H=0 and T=1  


So 010 = HTH  


The matrix forms a diagonal matrix that resembles some stairsteps

In [335]:
int('3')

3

In [336]:
#function to translate decimal to an arbritrary base

def baseb(n, b):
  if type(n) is str:
    n = int(n)
  e = n//b
  q = n%b
  if n == 0:
      return '0'
  elif e == 0:
      return str(q)
  else:
      return baseb(e, b) + str(q)

baseb(4, 2)

'100'

In [297]:
#generating markov matrix

def make_markov(mtx_length, base=base):

  #get identity/2
  step1 = np.eye(mtx_length//base, dtype=float)/base

  #interweave columns of identity/2 with itself
  step2 = np.zeros((step1.shape[0], step1.shape[0]*base))
  for i in range(base):
    step2[:,i::base] = step1 #interweaving

  #append a copy to the bottom
  step3 = step2.copy()
  for _ in range(base-1):
    step3 = np.append(step3,step2, axis=0)

  #done
  return step3


def get_states(mtx_length, base=2):
  states = {}
  for i in range(mtx_length):
    states[baseb(i, base)] = i
  return states


#length of subset of interest
length = 2
base = 3
mtx_length = base**length #side length of matrix


matrix = make_markov(mtx_length, base)

#displaying markov matrix in a dataframe
matrix_df = pd.DataFrame(matrix)

matrix_df

,0,1,2,3,4,5,6,7,8
0,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333
3,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333
6,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333


It is to be interpreted that moving along the columns is player 1 and moving across the rows is player 2

In [305]:

#function to get the probabilities when comparing 2 strings
def get_probabilities(players, base = 2, length=3):
  '''
  players = list of binary strings like [0b1, 0b111]
  mtx_length = number of unique states considered = 2^string_length
  '''
  mtx_length = base**length

  states = get_states(mtx_length, base)
  base_states = list(states.keys())
  unique_players = list(set(players))

  #modifying matrix with ending conditions, the 2 states considered
  I = np.eye(mtx_length,mtx_length, dtype = float)

  mtx_n = make_markov(mtx_length, base=base)

  player_states = [states[i] for i in unique_players] #binary to decimal

  for j in player_states:
    mtx_n[j,:] = I[j,:]


  #each state is equally likely
  start = np.ones((1,mtx_length))/mtx_length

  #we want the markov matrix to converge with repeat multiplications
  #100 is close enough when we round the answer
  n = 50
  M = mtx_n
  for _ in range(n):
    M = np.dot(M,M)


  #P = markov matrix
  #winnings = [0.125,...,0.125]*P^n, as n -> infinity
  winnings = np.dot(start,M.round(5)).round(5)

  return [winnings[:,i][0] for i in player_states]

base = 3
length = 2

get_probabilities(['0', '1','11'], base, length)

[0.36364, 0.27273, 0.36364]

Here, the output of get_probabilities comes out as (P(player 1 wins), P(player 2 wins))

#Getting the probabilities for each state

the code below works out every type of pairing and stores them all into a dataframe

In [310]:
base = 3
length = 2

all_probablities = []
states = get_states(mtx_length, base = base)
base_states = list(states.keys())


for row in range(mtx_length):
  temp = []
  for col in range(mtx_length):
    temp.append(get_probabilities([base_states[col], base_states[row]], 
                                  base = base, 
                                  length = length))
  all_probablities.append(temp)

winning_probablities = pd.DataFrame(all_probablities)

In [340]:
num_players = 2
base=4
length=2
mtx_length = base**length

states = get_states(mtx_length, base=base)
base_states = list(states.keys())
new_shape = list(it.repeat(mtx_length, num_players))#+[num_players]

#preparing list to hold all probabilities
all_probablities = list(range(mtx_length**num_players)) 

i = 0
for j in it.product(base_states, repeat=num_players):
  #print(get_probabilities(i))
  all_probablities[i] = get_probabilities(j, base=base, length=length)
  i += 1
  
#all_probablities
#all_probabilities = 
pd.DataFrame(np.reshape(np.array(all_probablities, dtype=object), new_shape))

#pd.DataFrame(all_probablities)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.375, 0.625]","[0.5, 0.5]","[0.44444, 0.55556]","[0.44444, 0.55556]","[0.375, 0.625]","[0.44444, 0.55556]","[0.5, 0.5]","[0.44444, 0.55556]","[0.375, 0.625]","[0.44444, 0.55556]","[0.44444, 0.55556]","[0.5, 0.5]"
1,"[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.375, 0.625]","[0.57143, 0.42857]","[0.57143, 0.42857]","[0.42857, 0.57143]","[0.5, 0.5]","[0.55556, 0.44444]","[0.5, 0.5]","[0.42857, 0.57143]","[0.5, 0.5]","[0.5, 0.5]","[0.55556, 0.44444]"
2,"[0.5, 0.5]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.42857, 0.57143]","[0.55556, 0.44444]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.57143, 0.42857]","[0.625, 0.375]","[0.57143, 0.42857]","[0.42857, 0.57143]","[0.5, 0.5]","[0.5, 0.5]","[0.55556, 0.44444]"
3,"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]",[1.0],"[0.42857, 0.57143]","[0.55556, 0.44444]","[0.5, 0.5]","[0.5, 0.5]","[0.42857, 0.57143]","[0.5, 0.5]","[0.55556, 0.44444]","[0.5, 0.5]","[0.5, 0.5]","[0.57143, 0.42857]","[0.57143, 0.42857]","[0.625, 0.375]"
4,"[0.375, 0.625]","[0.5, 0.5]","[0.42857, 0.57143]","[0.42857, 0.57143]",[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.42857, 0.57143]","[0.44444, 0.55556]","[0.5, 0.5]","[0.5, 0.5]","[0.57143, 0.42857]","[0.5, 0.5]","[0.55556, 0.44444]"
5,"[0.5, 0.5]","[0.375, 0.625]","[0.44444, 0.55556]","[0.55556, 0.44444]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.44444, 0.55556]","[0.375, 0.625]","[0.5, 0.5]","[0.44444, 0.55556]","[0.44444, 0.55556]","[0.375, 0.625]","[0.44444, 0.55556]","[0.5, 0.5]"
6,"[0.44444, 0.55556]","[0.57143, 0.42857]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.42857, 0.57143]","[0.5, 0.5]","[0.375, 0.625]","[0.42857, 0.57143]","[0.5, 0.5]","[0.57143, 0.42857]","[0.5, 0.5]","[0.55556, 0.44444]"
7,"[0.44444, 0.55556]","[0.57143, 0.42857]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.57143, 0.42857]","[0.44444, 0.55556]","[0.5, 0.5]","[0.57143, 0.42857]","[0.5, 0.5]","[0.42857, 0.57143]","[0.625, 0.375]"
8,"[0.375, 0.625]","[0.57143, 0.42857]","[0.5, 0.5]","[0.42857, 0.57143]","[0.5, 0.5]","[0.44444, 0.55556]","[0.42857, 0.57143]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.42857, 0.57143]","[0.55556, 0.44444]"
9,"[0.44444, 0.55556]","[0.5, 0.5]","[0.57143, 0.42857]","[0.5, 0.5]","[0.42857, 0.57143]","[0.375, 0.625]","[0.5, 0.5]","[0.42857, 0.57143]","[0.5, 0.5]",[1.0],"[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.57143, 0.42857]","[0.55556, 0.44444]"


#Finding the winning strategy

The winning strategy for player 1 can be found by looking at every option player 2 has and finding the string that beats it.



In [196]:
i_win = []

for col in range(mtx_length): #each col is a string player 2 could play
  temp = None
  for row in range(mtx_length): #each row is a string player 1 could play
    prob = winning_probabilities.iat[row, col]
  
    if temp is None and prob[0] != 1:
      temp = [prob, row, col]
    elif np.sum(prob[0]) >= 1:
      pass
    elif temp[0][0] < prob[0]:
      temp = [prob, row, col]

  i_win.append([f'{bin_states[temp[1]]} beats {bin_states[temp[2]]}',temp[0]])

#i_win_df = pd.DataFrame(i_win).T

i_win


NameError: ignored

If there is a string that is greater than 50%, then player 1 will win most of the time if going second in response to observing player 2's string

#Verifying the math
The probabilities above are made theoretically, so it would be nice to see it it can be observed

In [326]:

def list2str(array, base=2):
  '''
  Translate the random walk into a sequence of decimal to encode the state
  Input:
  array = 2d matrix of only 0's and 1's
  '''
  decimals = []

  if len(array.shape) == 1:
    num_flips = array.shape[0]
    num_trials = 1
  else:
    num_flips = array.shape[1]
    num_trials = array.shape[0]

  for trial in range(num_trials):
    walk = []
    for flip in range(3,num_flips+1):
      subwalk = int(''.join(map(str,array[trial,(flip-3):flip])),base)
      #subwalk = '0b'+''.join(map(str,array[trial,(flip-3):flip]))
      walk.append(subwalk)
    decimals.append(walk)

  return np.array(decimals)

def check_str(players, array, base = 2, length=3):
  '''
  Input:
  players=list of binary strings
  array=decimal translating of random walk per subwalk
  string_length=length of string used for the game
  '''
  mtx_length = base**length
  states = get_states(mtx_length, base=base) #binary to decimal
  base_states = list(states.keys()) #decimal to binary
  unique_players = list(set(players)) #removes duplicates

  #make a dictionary from list of strings
  count = dict.fromkeys(unique_players,0)

  for row in range(array.shape[0]):
    for col in range(array.shape[1]):
      sub_state = base_states[array[row,col]] #decimal to binary
      if sub_state in unique_players:
        count[sub_state] += 1
        break
      else:
        assert 'Unknown Error'

  total = np.sum(list(count.values()))

  for i in unique_players:
    count[i] = count[i]/total
  return count


num_flips = 50
num_trials = 10

base = 3
length = 3

walk = np.random.randint(0,base,(num_trials,num_flips))

print(check_str(['0','1'], 
                array = list2str(walk, base=base), 
                base = base, 
                length=length))



{'0': 0.3, '1': 0.7}


Testing out a few comparison seems to agree with theory, nice!

In [ ]:
get_states(10).values()

In [204]:
players=['0b0','0b1','0b1']
list(set(players))



['0b1', '0b0']